# Exercício 1 Aula 6

## Batch Normalization

Durante o treino, batch normalization calcula a média μ B e desvio-padrão σ B do batch B. No caso de
imagens, são calculadas média e desvio-padrão para cada banda de cor (ou para cada saída do fil-
tro). Depois, normaliza-se o batch subtraindo μ B e dividindo por σ B +ε cada elemento de B, onde ε
no denominador é um número pequeno para evitar divisão por zero. A saída do batch normalization
é uma distribuição normal de média zero e desvio-padrão um. Nem sempre esta distribuição é ade-
quada para alimentar a camada seguinte. Consequentemente, a saída de batch normalization é “des-
normalizada” multiplicando por um parâmetro “desvio padrão” γ (inicializada com 1) e somando
um parâmetro “média” β (inicializada com 0).


$$B = (B - μ B ) / (σ B +ε)  γ + β$$

Onde B são os elementos do batch. Os parâmetros ótimos γ e β são atualizados pela retro-propaga-
ção.
Durante a predição, são utilizadas média μ B , desvio σ B , γ e β calculados no treino para normalizar o
batch. Porém, μ B e σ B são atualizados durante a predição, para se ajustar à estatística dos dados de
teste:

$$B = (B - moving_μ B ) / (moving_σ B +ε) * γ + β$$

In [14]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Flatten, BatchNormalization
from keras import optimizers
import numpy as np

In [15]:
batch_size = 100
num_classes = 10
epochs = 5

In [16]:
nl, nc = 32,32
(ax, ay), (qx, qy) = cifar10.load_data()

In [17]:
ax = ax.reshape(ax.shape[0], nl, nc, 3)
qx = qx.reshape(qx.shape[0], nl, nc, 3)
input_shape = (nl, nc, 3)

In [18]:
ax= ax.astype('float32')
qx= qx.astype('float32')

ax/= 255 #0 a 1
qx/= 255 #0 a 1

In [19]:
ay = keras.utils.to_categorical(ay, num_classes)
qy = keras.utils.to_categorical(qy, num_classes)

In [20]:
model = Sequential()

# Added BatchNormalization no começo
model.add(BatchNormalization(input_shape=input_shape))

model.add(Conv2D(20, kernel_size=(3,3), activation='relu', padding='same', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2))) #16x16x20

model.add(Conv2D(40, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2))) #8x8x40

model.add(Conv2D(80, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2))) #4x4x80

model.add(Conv2D(160, kernel_size=(3,3), activation='relu', padding='same')) #4x4x160

model.add(Flatten())

model.add(Dense(1000,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

In [21]:
from keras.utils import print_summary
print_summary(model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 20)        560       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 40)        7240      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 40)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 80)          28880     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 80)          0         
__________

In [22]:
opt=optimizers.Adam()

model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
filters = model.get_layer(index=0).get_weights()
print(filters)

[array([1., 1., 1.], dtype=float32), array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32), array([1., 1., 1.], dtype=float32)]


In [24]:
model.fit(ax, ay, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(qx, qy))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
 - 79s - loss: 1.3481 - acc: 0.5118 - val_loss: 1.1135 - val_acc: 0.6064
Epoch 2/5
 - 79s - loss: 0.9209 - acc: 0.6746 - val_loss: 0.9343 - val_acc: 0.6750
Epoch 3/5
 - 77s - loss: 0.7296 - acc: 0.7412 - val_loss: 0.7970 - val_acc: 0.7239
Epoch 4/5
 - 80s - loss: 0.5918 - acc: 0.7906 - val_loss: 0.7857 - val_acc: 0.7336
Epoch 5/5
 - 100s - loss: 0.4758 - acc: 0.8313 - val_loss: 0.8363 - val_acc: 0.7327


In [25]:
filters = model.get_layer(index=0).get_weights()
print(filters)

[array([1.0070899, 1.0843607, 1.0422177], dtype=float32), array([0.02914334, 0.05872931, 0.02690289], dtype=float32), array([0.4911057 , 0.48164928, 0.44525656], dtype=float32), array([0.06089546, 0.0591564 , 0.06806222], dtype=float32)]


Looking at the code : https://github.com/keras-team/keras/blob/master/keras/layers/normalization.py#L61-L72

I think it should be [gamma, beta, mean , variance]

<br>
<br>

$\gamma = [1.0070899, 1.0843607, 1.0422177]$
<br>
<br>

$\beta = [0.02914334, 0.05872931, 0.02690289]$
<br>
<br>

$\mu = [0.4911057 , 0.48164928, 0.44525656]$
<br>
<br>

$\sigma = [0.06089546, 0.0591564 , 0.06806222]$

Onde cada um dos três valores equivale ao gamma, beta, mu e sigma de cada uma das bandas

In [26]:
score = model.evaluate(qx, qy, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 0.8363285460472107)
('Test accuracy:', 0.7327)
